In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../src/')  # point to your modules if not in the same dir

from scraper.session import login
from scraper.fetch import get_court_links

session = login()
links = get_court_links(session)
print(links)



Login successful
getting court links
1
now??
